# Background scales and subtraction for $xy$-like data files in $Q$-space

May, 2023  
Martin Aaskov Karlsen  
Postdoc  
Ravnsbæk Group  
Aarhus University  

This iPython notebook is meant for background subtraction for $xy$-like files in
$Q$-space, i.e., reciprocal space.  
The data files and the background file should
be place in a folder, e.g., `chi`.  
The user will need to specify the name of the folder containing the files 
together with the name of the background file.

Background scale factors will be saved to a .txt file.  
Background-subtracted data files will be written and will be plotted.

Imports.

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

Specify path for directory containing data files and background file.

In [ ]:
data_path = Path.cwd() / "chi"

Specify name of background file.

In [ ]:
bkg_file = "DBR_001_600s_0abs_SDD290mm_sum_9070_9079_0.chi"
bkg_path = data_path / bkg_file

Collecting data files.

In [ ]:
data_files = list(data_path.glob("*.chi"))
data_files = [f for f in data_files if not f.name == bkg_file]

Function to obtain scale maximum scale factor for background resulting in no
negative intensity values for the background-subtracted data.

In [ ]:
def bkg_scale(file, bkg):
    x, y = np.loadtxt(file).T
    x_bkg, y_bkg = np.loadtxt(bkg).T
    scale = 1
    ydiff = y - (scale * y_bkg)
    for i in range(1, 11):
        scale_step = 10**-i
        upscale, downscale = False, False
        if np.any(ydiff < 0):
            downscale = True        
        else:
            upscale = True
        while upscale:
            scale += scale_step
            ydiff = y - (scale * y_bkg)
            if np.any(ydiff < 0):
                upscale = False
                scale -= scale_step
            else:
                upscale = True

        while downscale:
            scale -= scale_step
            ydiff = y - (scale * y_bkg)
            if np.any(ydiff < 0):
                downscale = True
            else:
                downscale = False

    return scale

Function for subtracting background.

In [ ]:
def bkg_subtraction(data_file, bkg_file, scale, outputpath):
    x, y = np.loadtxt(data_file).T
    x_bkg, y_bkg = np.loadtxt(bkg_file).T
    y_sub = y - (scale * y_bkg)
    outputpath_full = outputpath / f"{data_file.stem}_bkgsub{data_file.suffix}"
    np.savetxt(outputpath_full,
               np.column_stack((x, y_sub)),
               delimiter="\t",
               encoding="utf8",
               )
    
    return np.column_stack((x, y_sub))

Function for plotting the background-subtracted data.

In [ ]:
def plot(xy_sub, name, output_paths):
    dpi = 600
    figsize = (12, 4)
    fontsize_labels = 20
    fontsize_ticks = 14
    xlabel = "$Q\;[\mathrm{\AA}^{-1}]$"
    ylabel = "$I\;[\mathrm{arb.\;u.}]$"
    x, y = xy_sub.T
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(x, y)
    ax.set_xlim(np.amin(x), np.amax(x))
    ax.set_xlabel(xlabel, fontsize=fontsize_labels)
    ax.set_ylabel(ylabel, fontsize=fontsize_labels)
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.tick_params(axis="both", labelsize=fontsize_ticks)
    ax.minorticks_on()
    for p in output_paths:
        print(f"\t\t{p.name}")
        plt.savefig(p / f"{name}.{p.name}", bbox_inches="tight", dpi=dpi)
    plt.close()

    return None

Creating output paths if not already existing.

In [ ]:
txt_path = Path.cwd() / "txt"
bkgsub_path = Path.cwd() / f"{data_path.name}_bkgsub"
for p in [txt_path, bkgsub_path]:
    if not p.exists():
        p.mkdir()
plot_folders = ["png", "pdf", "svg"]
plot_paths = [Path.cwd() / folder for folder in plot_folders]
for p in plot_paths:
    if not p.exists():
        p.mkdir()

Obtaining and collecting scale factors, writing background-subtracted data files
and plotting the background-subtracted data.

In [ ]:
scales = []
for i, f in enumerate(data_files):
    print(f"file {i+1}\n\t{f.name}")
    scale = bkg_scale(f, bkg_path)
    print(f"\tscale:\t{scale:.10f}")
    scales.append(scale)
    xy_sub = bkg_subtraction(f, bkg_path, scale, bkgsub_path)
    plot(xy_sub, f.stem, plot_paths)

Collecting scale factors and file names, making string with these, and writing
to .txt file.

In [ ]:
idx_scale_name = list(zip(range(len(scales)), 
                          scales, 
                          [f.name for f in data_files],
                          ))
s = "index\tscale\t\tfilename\n"
for e in idx_scale_name:
    idx, scale, name = e
    s += f"{idx}\t{scale}\t{name}\n"
output_path = txt_path / "bkg_scales.txt"
with output_path.open(mode="w", encoding="utf-8") as o:
    o.write(s)